<a href="https://colab.research.google.com/github/chongzicbo/Dive-into-Deep-Learning-tf.keras/blob/master/3.13.%20%E4%B8%A2%E5%BC%83%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##3.13. 丢弃法
除了前一节介绍的权重衰减以外，深度学习模型常常使用丢弃法（dropout）[1] 来应对过拟合问题。丢弃法有一些不同的变体。本节中提到的丢弃法特指倒置丢弃法（inverted dropout）。

###3.13.1. 方法
回忆一下，“多层感知机”一节的图3.3描述了一个单隐藏层的多层感知机。其中输入个数为4，隐藏单元个数为5，且隐藏单元 $h_i$
$i=1, \ldots, 5$的计算表达式为$$
h_i = \phi\left(x_1 w_{1i} + x_2 w_{2i} + x_3 w_{3i} + x_4 w_{4i} + b_i\right),
$$
这里$\phi$是激活函数，$x_1, \ldots, x_4$是输入，隐藏单元$i$的权重参数为$w_{1i}, \ldots, w_{4i}$,偏差参数为$b_i$。当对该隐藏层使用丢弃法时，该层的隐藏单元将有一定概率被丢弃掉。设丢弃概率为 $p$ ， 那么有 $p$ 的概率 $h_i$ 会被清零，有 1-$p$ 的概率 $h_i$ 会除以 1−$p$ 做拉伸。丢弃概率是丢弃法的超参数。具体来说，设随机变量$\xi_i$为0和1的概率分别为 $p$ 和 1−$p$ 。使用丢弃法时我们计算新的隐藏单元 $h^′_i$
$$
h_i' = \frac{\xi_i}{1-p} h_i.
$$
由于$E(\xi_i) = 1-p$,因此
$$
E(h_i') = \frac{E(\xi_i)}{1-p}h_i = h_i.
$$

即丢弃法不改变其输入的期望值。让我们对图3.3中的隐藏层使用丢弃法，一种可能的结果如图3.5所示，其中 $h_2$ 和 $h_5$ 被清零。这时输出值的计算不再依赖 $h_2$ 和 $h_5$ ，在反向传播时，与这两个隐藏单元相关的权重的梯度均为0。由于在训练中隐藏层神经元的丢弃是随机的，即 $h_1, \ldots, h_5$ 都有可能被清零，输出层的计算无法过度依赖 $h_1, \ldots, h_5$ 中的任一个，从而在训练模型时起到正则化的作用，并可以用来应对过拟合。在测试模型时，我们为了拿到更加确定性的结果，一般不使用丢弃法。
<div align=center><img src="https://zh.gluon.ai/_images/dropout.svg" width="300"/></div>
<center>图 3.5 隐藏层使用了丢弃法的多层感知机</center>

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import losses
from tensorflow.keras import layers,Sequential
import numpy as np
import tensorflow.data as tfdata

In [0]:
tf.enable_eager_execution()

###3.13.2. 从零开始实现
根据丢弃法的定义，我们可以很容易地实现它。下面的dropout函数将以drop_prob的概率丢弃输入X中的元素。

In [0]:
def dropout(X,drop_prob):
  assert 0<= drop_prob<=1
  keep_prob=1-drop_prob
  #这种情况下把全部元素丢弃
  if keep_prob==0:
    return tf.zeros_like(X)
  # mask=tf.random.uniform(shape=(tf.shape(X)),minval=0,maxval=1).numpy()<keep_prob
  mask=tf.less(tf.random.uniform(shape=(tf.shape(X)),minval=0,maxval=1),y=tf.constant(keep_prob,dtype=tf.float32))
  return tf.cast(mask,dtype=tf.float32) * X/keep_prob

我们运行几个例子来测试一下dropout函数。其中丢弃概率分别为0、0.5和1。

In [0]:
X=tf.reshape(tf.range(16,dtype=tf.float32),shape=(2,8))
print(X)
dropout(X,0.0)

tf.Tensor(
[[ 0.  1.  2.  3.  4.  5.  6.  7.]
 [ 8.  9. 10. 11. 12. 13. 14. 15.]], shape=(2, 8), dtype=float32)


<tf.Tensor: id=18, shape=(2, 8), dtype=float32, numpy=
array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11., 12., 13., 14., 15.]], dtype=float32)>

In [0]:
dropout(X,0.5)

<tf.Tensor: id=31, shape=(2, 8), dtype=float32, numpy=
array([[ 0.,  0.,  4.,  6.,  0.,  0., 12., 14.],
       [16., 18., 20., 22., 24., 26.,  0., 30.]], dtype=float32)>

In [0]:
dropout(X,1)

<tf.Tensor: id=32, shape=(2, 8), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

####3.13.2.1. 定义模型参数
实验中，我们依然使用“softmax回归的从零开始实现”一节中介绍的Fashion-MNIST数据集。我们将定义一个包含两个隐藏层的多层感知机，其中两个隐藏层的输出个数都是256。

In [0]:
num_inputs,num_outputs,num_hiddens1,num_hiddens2=784,10,256,256

W1=tf.Variable(tf.random.normal(shape=(num_inputs,num_hiddens1),stddev=0.01))
b1=tf.Variable(tf.zeros(num_hiddens1))

W2=tf.Variable(tf.random.normal(shape=(num_hiddens1,num_hiddens2),stddev=0.01))
b2=tf.Variable(tf.zeros(num_hiddens2))

W3=tf.Variable(tf.random.normal(shape=(num_hiddens2,num_outputs),stddev=0.01))
b3=tf.Variable(tf.zeros(num_outputs))

params=[W1,b1,W2,b2,W3,b3]

####3.13.2.2. 定义模型
下面定义的模型将全连接层和激活函数ReLU串起来，并对每个激活函数的输出使用丢弃法。我们可以分别设置各个层的丢弃概率。通常的建议是把靠近输入层的丢弃概率设得小一点。在这个实验中，我们把第一个隐藏层的丢弃概率设为0.2，把第二个隐藏层的丢弃概率设为0.5。我们可以通过is_training参数来判断运行模式为训练还是测试，并只需在训练模式下使用丢弃法。

In [0]:
drop_prob1,drop_prob2=0.2,0.5
def net(X,is_training=True):
  # X=tf.reshape(X,shape=(-1,num_inputs))
  H1=keras.activations.relu(tf.matmul(X,W1)+b1)
  if is_training:
    H1=dropout(H1,drop_prob1)
  H2=keras.activations.relu(tf.matmul(H1,W2)+b2)
  if is_training:
    H2=dropout(H2,drop_prob2)
  return tf.matmul(H2,W3)+b3

####3.13.2.3. 训练和测试模型
这部分与之前多层感知机的训练和测试类似。

In [0]:
batch_size=256
buffer_size=10000
def load_data_fashion_mnist(batch_size,buffer_size):
  (x_train,y_train),(x_test,y_test)=keras.datasets.fashion_mnist.load_data()
  x_train=np.reshape(x_train,[x_train.shape[0],-1])#将三维张量reshape成二维
  x_test=np.reshape(x_test,[x_test.shape[0],-1])
  train_iter=tfdata.Dataset.from_tensor_slices((x_train,y_train)).map(lambda x,y:(x/255,y)).shuffle(buffer_size).batch(batch_size)
  test_iter=tfdata.Dataset.from_tensor_slices((x_test,y_test)).map(lambda x,y:(x/255,y)).batch(batch_size)
  return train_iter,test_iter

In [0]:
def sgd(params,loss,t,lr,batch_size):
  for param in params:
    dl_dp=t.gradient(loss,param) #求梯度
    param.assign_sub(lr*dl_dp/batch_size) #更新梯度

def evaluate_accuracy(data_iter,net):
  acc_sum,n=0.0,0
  for X,y in data_iter:
    # y=tf.cast(y,tf.float32)
    acc_sum+=tf.equal(tf.cast(tf.argmax(net(X,is_training=False),axis=1),tf.float32),tf.cast(y,tf.float32)).numpy().sum()
    n+=tf.shape(y)[0].numpy()
  return acc_sum/n    
def train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,params=None,lr=None,trainer=None):
  for epoch in range(num_epochs):
    train_l_sum,train_acc_sum,n=0.0,0.0,0
    for X,y in train_iter:
      if trainer is None:
        with tf.GradientTape(persistent=True) as t:
          y_hat=net(X)
          l=tf.reduce_sum(loss(y,y_hat))
        sgd(params,l,t,lr,batch_size)
      else:
        y_hat=net(X)
        l=tf.reduce_sum(loss(y_hat,y))
        trainer.minimize(lambda:loss(net(X),y),global_step=tf.train.get_or_create_global_step())
      train_l_sum+=l.numpy()
      train_acc_sum+=tf.equal(tf.cast(tf.argmax(y_hat,axis=1),tf.float32),tf.cast(y,tf.float32)).numpy().sum()
      n+=tf.shape(y)[0]
    test_acc=evaluate_accuracy(test_iter,net)
    print('epoch %d,loss %.4f,train acc %.3f,test acc %.3f'%(epoch+1,train_l_sum/n,train_acc_sum/n,test_acc))  

In [0]:
num_epochs,lr,batch_size=10,0.5,256
buffer_size=10000
loss=losses.SparseCategoricalCrossentropy(from_logits=True)
train_iter,test_iter=load_data_fashion_mnist(batch_size=batch_size,buffer_size=buffer_size)
train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,params=params,lr=lr)

###3.13.3. 简洁实现
在tf.keras中，我们只需要在全连接层后添加Dropout层并指定丢弃概率。在训练模型时，Dropout层将以指定的丢弃概率随机丢弃上一层的输出元素；在测试模型时，Dropout层并不发挥作用。

In [0]:
train_iter,test_iter=load_data_fashion_mnist(batch_size=batch_size,buffer_size=buffer_size)
net=Sequential()
net.add(layers.Dense(256,activation='relu'))
net.add(layers.Dropout(0.5))
net.add(layers.Dense(256,activation='relu'))
net.add(layers.Dropout(0.5))
net.add(layers.Dense(10))
net.compile(optimizer=keras.optimizers.SGD(learning_rate=0.05),loss=loss,metrics=['accuracy'])
net.fit_generator(train_iter,validation_data=test_iter,epochs=num_epochs)

Epoch 1/10
235/235 [==============================] - 11s 48ms/step - loss: 1.1370 - acc: 0.4705 - val_loss: 0.6734 - val_acc: 0.7585
Epoch 2/10
235/235 [==============================] - 11s 46ms/step - loss: 0.7373 - acc: 0.7231 - val_loss: 0.5757 - val_acc: 0.7936
Epoch 3/10
235/235 [==============================] - 11s 47ms/step - loss: 0.6395 - acc: 0.7695 - val_loss: 0.5189 - val_acc: 0.8137
Epoch 4/10
235/235 [==============================] - 11s 48ms/step - loss: 0.5891 - acc: 0.7875 - val_loss: 0.4930 - val_acc: 0.8201
Epoch 5/10
235/235 [==============================] - 11s 46ms/step - loss: 0.5517 - acc: 0.8016 - val_loss: 0.4723 - val_acc: 0.8271
Epoch 6/10
235/235 [==============================] - 11s 46ms/step - loss: 0.5260 - acc: 0.8167 - val_loss: 0.4573 - val_acc: 0.8323
Epoch 7/10
235/235 [==============================] - 11s 46ms/step - loss: 0.5079 - acc: 0.8200 - val_loss: 0.4449 - val_acc: 0.8371
Epoch 8/10
235/235 [==============================] - 11s 47ms

###3.13.4. 小结
* 我们可以通过使用丢弃法应对过拟合。
* 丢弃法只在训练模型时使用。